In [1]:
import tensorflow as tf

In [20]:
class MCC:
    
    def __init__(self,filters,act='relu',cact='sigmoid',c=1,input_shape=(256,256,3),strides=(2,2),kernel=(3,3),kreg=None):
    
        self.filters = filters ;
        self.act = act ;
        self.input_shape = input_shape ;
        self.strides = strides
        self.kernel = kernel ;
        self.cact = cact
        self.c = c ;
        
        if kreg==None:
            self.kreg = None;
        else:
            self.kreg = kreg

        return


    def Classifier(self):
        inp_layer = tf.keras.layers.Input(shape=self.input_shape,name='encoder input')
        x = inp_layer;

        for filter in self.filters:
            x = tf.keras.layers.Conv2D(filter,kernel_size=self.kernel,strides=self.strides,activation=self.act,kernel_regularizer=self.kreg)(x)

        latent = tf.keras.layers.Flatten(name='latent_flatten')(x)
        latent = tf.keras.layers.Dense(self.c,activation=self.cact)(latent)
        classifier = tf.keras.Model(inputs=inp_layer,outputs=latent,name='Classifier')

        return classifier


    def Classifier_plus(self,neurons):
        self.neurons = neurons; 
        inp_layer = tf.keras.layers.Input(shape=self.input_shape,name='encoder input')
        x = inp_layer;

        for filter in self.filters:
            x = tf.keras.layers.Conv2D(filter,kernel_size=self.kernel,strides=self.strides,activation=self.act,kernel_regularizer=self.kreg)(x)

        latent = tf.keras.layers.Flatten(name='latent_flatten')(x)
        x2 = latent ;
        
        for neuron in self.neurons:
            x2 = tf.keras.layers.Dense(neuron,activation=self.act,kernel_regularizer=self.kreg)(x2)
        
        x2 = tf.keras.layers.Dense(self.c,activation=self.cact)(x2)
        
        classifier = tf.keras.Model(inputs=inp_layer,outputs=x2,name='Classifier2')

        return classifier

In [21]:
mcc = MCC([32,64],act='selu',c=4,cact='softmax')

In [22]:
c1 = mcc.Classifier()

In [23]:
c1.summary()

Model: "Classifier"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 encoder input (InputLayer)  [(None, 256, 256, 3)]     0         
                                                                 
 conv2d_12 (Conv2D)          (None, 127, 127, 32)      896       
                                                                 
 conv2d_13 (Conv2D)          (None, 63, 63, 64)        18496     
                                                                 
 latent_flatten (Flatten)    (None, 254016)            0         
                                                                 
 dense_8 (Dense)             (None, 4)                 1016068   
                                                                 
Total params: 1,035,460
Trainable params: 1,035,460
Non-trainable params: 0
_________________________________________________________________


In [24]:
c2 = mcc.Classifier_plus([500,100])

In [25]:
c2.summary()

Model: "Classifier2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 encoder input (InputLayer)  [(None, 256, 256, 3)]     0         
                                                                 
 conv2d_14 (Conv2D)          (None, 127, 127, 32)      896       
                                                                 
 conv2d_15 (Conv2D)          (None, 63, 63, 64)        18496     
                                                                 
 latent_flatten (Flatten)    (None, 254016)            0         
                                                                 
 dense_9 (Dense)             (None, 500)               127008500 
                                                                 
 dense_10 (Dense)            (None, 100)               50100     
                                                                 
 dense_11 (Dense)            (None, 4)                 